# Hand Pick Turbine Locations

In [ ]:
%matplotlib inline

import os
import sys
import glob
import time
import numpy as np
import scipy
import scipy.interpolate
import matplotlib
import matplotlib.pyplot as plt

# Local modules
import toshiba
import cinterp

In [ ]:
zmap = cinterp.colormap.zmap()
cmap = matplotlib.colors.LinearSegmentedColormap.from_list('colors', zmap[:, :3], N=len(zmap))

In [ ]:
files = glob.glob(os.path.expanduser('~/Downloads/toshiba/*.nc'))
filename = files[0]

# azimuth_count = 360
range_count = 200

In [ ]:
sweep = toshiba.read(filename, maxgate=range_count)

In [ ]:
plt.figure(dpi=200)
plt.pcolormesh(sweep['z'], cmap=cmap)
plt.clim((-32, 96))
plt.colorbar()
plt.grid()

In [ ]:
# Get the reflectivity and raw range and azimuth angles
z = sweep['z']
r = 1.0e-3 * sweep['rr']
a = sweep['aa']

# Only retain the 360-deg coverage, take out the extra piece
k = next(k for k, ia in enumerate(a[::-1]) if abs(ia > a[0]) < 0.5) 
a = a[:-k] / 180.0 * np.pi
z = sweep['z'][:-k, :]

rr, aa = np.meshgrid(r, a)
xx = rr * np.sin(aa)
yy = rr * np.cos(aa)

In [ ]:
# cinterp.data = importlib.reload(cinterp.data)

In [ ]:
turbines = cinterp.data.toshiba_cells

a_turb = turbines[:, 0] / 180.0 * np.pi
r_turb = turbines[:, 1]
x_turb = r_turb * np.sin(a_turb)
y_turb = r_turb * np.cos(a_turb)

xlim = (-1, 6)
ylim = (-12.5, -5.5)
# xlim = (2.2, 4.2)
# ylim = (-10.5, -8.5)

plt.figure(figsize=(5.25, 4), dpi=200)
plt.pcolormesh(xx, yy, z, cmap=cmap)
plt.plot(x_turb, y_turb, 'xw')
plt.clim((-32, 96))
plt.colorbar()
plt.grid()
# plt.xlim((-5, 5))
# plt.ylim((-14, -4))
# plt.xlim((0, 5))
# plt.ylim((-10.5, -5.5))
plt.xlim(xlim)
plt.ylim(ylim)

In [ ]:
cells = cinterp.pos2cellid(turbines, a / np.pi * 180.0, r)

In [ ]:
# for c in cells:
#     print((a[c[0]] * 180.0 / np.pi, r[c[1]]))

In [ ]:
z_interp = z.copy()

m = np.zeros(z_interp.shape, dtype=bool)
for c in cells:
    m[c[0], c[1]] = True
m1 = cinterp.dilate(m)
m2 = cinterp.dilate(m1)
    
z_interp[m2] = 36
z_interp[m] = 66.0

In [ ]:
plt.figure(figsize=(5.25, 4), dpi=200)
plt.pcolormesh(xx, yy, z_interp, cmap=cmap)
plt.plot(x_turb, y_turb, 'xw')
plt.clim((-32, 96))
plt.colorbar()
plt.grid()
plt.xlim(xlim)
plt.ylim(ylim)

In [ ]:
c = cinterp.mask2tags(m1)

# Take out the dilated portion
c[m1 ^ m] = 0

In [ ]:
plt.figure(figsize=(5.25, 4), dpi=200)
plt.pcolormesh(xx, yy, 5 * c, cmap=cmap)
plt.clim((-32, 96))
plt.colorbar()
plt.grid()
plt.xlim(xlim)
plt.ylim(ylim)